In [1]:
import sklearn
import pandas as pd 
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from functools import partial
from pathlib import Path
import os
from pandas import Timedelta

In [2]:
df_covid = pd.read_csv("../../data/covid/usa_urban_no_density_smoothed.csv")
df_covid['date'] = pd.to_datetime(df_covid['date'], infer_datetime_format=True)

In [3]:
# Période de 7 jours entre 3 et 10 jours avant la période étudiée de cas de COVID => 10 jours entre le début de la période CAMS et le début de la période COVID
delta = Timedelta(10,"day")

df_cams = pd.read_csv("../../data/cams_data.csv")
df_cams['time'] = pd.to_datetime(df_cams['time'], infer_datetime_format=True)
df_cams['time'] = df_cams['time'].apply(lambda x : x.date() + delta)

In [4]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34521 entries, 0 to 34520
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            34521 non-null  datetime64[ns]
 1   mobility_transit_stations       34521 non-null  float64       
 2   mobility_retail_and_recreation  34521 non-null  float64       
 3   mobility_grocery_and_pharmacy   34521 non-null  float64       
 4   mobility_parks                  34521 non-null  float64       
 5   mobility_residential            34521 non-null  float64       
 6   mobility_workplaces             34521 non-null  float64       
 7   county_id                       34521 non-null  int64         
 8   smoothed_cumul                  34521 non-null  float64       
 9   smoothed_d1                     34521 non-null  float64       
 10  smoothed_d2                     34521 non-null  float64       
dtypes:

In [5]:
df_covid

,date,mobility_transit_stations,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_residential,mobility_workplaces,county_id,smoothed_cumul,smoothed_d1,smoothed_d2
0,2020-03-29,-60.000000,-48.000000,-31.000000,5.000000,13.000000,-37.000000,10003,141.809524,21.892857,3.880952
1,2020-03-30,-49.000000,-39.000000,-25.000000,9.000000,20.000000,-50.000000,10003,166.000000,23.750000,2.785714
2,2020-03-31,-51.000000,-40.000000,-25.000000,-19.000000,22.000000,-52.000000,10003,191.000000,25.892857,2.642857
3,2020-04-01,-45.000000,-37.000000,-19.000000,23.000000,22.000000,-52.000000,10003,212.238095,38.250000,11.309524
4,2020-04-02,-49.000000,-37.000000,-20.000000,13.000000,23.000000,-53.000000,10003,254.857143,46.392857,11.071429
...,...,...,...,...,...,...,...,...,...,...,...
34516,2020-11-16,-24.000000,-20.000000,-3.000000,14.000000,13.000000,-31.000000,55133,20912.666667,777.357143,245.380952
34517,2020-11-17,-28.000000,-23.000000,-4.000000,25.000000,15.000000,-36.000000,55133,21829.952381,893.464286,179.880952
34518,2020-11-18,-28.041379,-22.313725,-3.843137,24.462366,14.792453,-35.274510,55133,23075.476190,906.571429,-96.523810
34519,2020-11-19,-28.082759,-21.627451,-3.686275,23.924731,14.584906,-34.549020,55133,24195.761905,770.071429,-322.095238


In [4]:
df_cams = df_cams.groupby(['time','shapeID']).agg([np.mean,np.min,np.max])
df_cams

x0                            x1            \
                        mean      amin      amax      mean      amin   
time       shapeID                                                     
2020-01-11 0        2.560990  1.132602  3.920752 -0.621472 -1.321482   
           1        1.585728  1.003614  2.163140 -1.442269 -2.552994   
           2        2.336001  0.740924  3.576501 -1.030909 -1.611304   
           3        2.061235  1.591767  2.767817 -1.015826 -2.157733   
           4        1.688767  1.123299  2.411143 -1.518019 -2.608174   
...                      ...       ...       ...       ...       ...   
2020-11-26 3228     2.863880  2.354470  3.442855 -0.367008 -1.288185   
           3229     2.193660  1.366086  3.337594 -1.029890 -1.669004   
           3230     2.997420  1.879952  4.827222  0.063381 -1.229616   
           3231     2.783941  1.074804  4.555070 -1.077723 -1.666085   
           3232     2.981951  1.493466  4.497534 -0.796010 -1.367780   

                                    x2                            x3  ...  \
                        amax      mean      amin      amax      mean  ...   
time       shapeID                                                    ...   
2020-01-11 0       -0.136410  0.547259 -0.205632  1.450366  0.328272  ...   
           1       -0.001904  0.082670 -1.015893  1.228365 -0.001149  ...   
           2       -0.252841  0.570662  0.064298  1.334669  0.299919  ...   
           3        0.369767  0.257655 -0.431713  1.232167  0.165409  ...   
           4       -0.092445  0.172981 -0.700243  1.326847  0.052826  ...   
...                      ...       ...       ...       ...       ...  ...   
2020-11-26 3228     1.237951 -1.062319 -1.936876  0.360785 -1.180580  ...   
           3229    -0.050964 -0.403719 -1.967603  0.662261 -0.703278  ...   
           3230     1.308801  0.369541 -0.660267  1.273194  0.033446  ...   
           3231    -0.397469 -0.420119 -1.787267  1.132561 -0.868802  ...   
           3232     0.058086 -0.259783 -1.216226  1.081014 -0.715025  ...   

                          x7        x8                            x9  \
                        amax      mean      amin      amax      mean   
time       shapeID                                                     
2020-01-11 0       -0.127221  0.663481  0.353241  1.333196  0.305105   
           1       -0.079008  0.848270  0.674095  1.125456  0.120760   
           2       -0.030143  0.827391  0.629278  1.069803  0.427520   
           3       -0.051850  0.786006  0.669407  0.988720  0.164188   
           4       -0.053965  0.943710  0.779539  1.154469  0.169794   
...                      ...       ...       ...       ...       ...   
2020-11-26 3228    -0.077145 -0.218845 -0.677157  0.241384  0.066385   
           3229    -0.655699  0.127605 -0.468057  0.828150 -0.740185   
           3230    -0.563107 -0.703884 -1.031925 -0.244547  0.364359   
           3231     0.145051  0.240206 -0.567698  1.125124 -0.909044   
           3232    -0.062040  0.352340 -0.322112  1.110727 -0.867497   

                                             x10                      
                        amin      amax      mean      amin      amax  
time       shapeID                                                    
2020-01-11 0       -0.023992  0.601407  0.710089 -0.072662  1.222776  
           1       -0.072434  0.449380  0.062689 -0.147292  0.270271  
           2        0.105998  0.901687  0.225709 -0.144999  0.514646  
           3       -0.042237  0.478838  0.284957  0.014918  0.596166  
           4       -0.014473  0.457357 -0.019206 -0.254867  0.203410  
...                      ...       ...       ...       ...       ...  
2020-11-26 3228    -0.106235  0.233775 -0.739699 -0.981184 -0.474419  
           3229    -0.993194 -0.458630  0.147552 -0.153186  0.917622  
           3230     0.033683  0.590167 -0.308732 -0.452348 -0.056429  
           3231    -1.071697 -0.420652 -0.160216 -0.552683  0.257796  
      

In [5]:
df_cams.columns = pd.Index([f"{x}_{y}" for (x,y) in df_cams.columns])
df_cams = df_cams.reset_index()

df_cams

,time,shapeID,x0_mean,x0_amin,x0_amax,x1_mean,x1_amin,x1_amax,x2_mean,x2_amin,...,x7_amax,x8_mean,x8_amin,x8_amax,x9_mean,x9_amin,x9_amax,x10_mean,x10_amin,x10_amax
0,2020-01-11,0,2.560990,1.132602,3.920752,-0.621472,-1.321482,-0.136410,0.547259,-0.205632,...,-0.127221,0.663481,0.353241,1.333196,0.305105,-0.023992,0.601407,0.710089,-0.072662,1.222776
1,2020-01-11,1,1.585728,1.003614,2.163140,-1.442269,-2.552994,-0.001904,0.082670,-1.015893,...,-0.079008,0.848270,0.674095,1.125456,0.120760,-0.072434,0.449380,0.062689,-0.147292,0.270271
2,2020-01-11,2,2.336001,0.740924,3.576501,-1.030909,-1.611304,-0.252841,0.570662,0.064298,...,-0.030143,0.827391,0.629278,1.069803,0.427520,0.105998,0.901687,0.225709,-0.144999,0.514646
3,2020-01-11,3,2.061235,1.591767,2.767817,-1.015826,-2.157733,0.369767,0.257655,-0.431713,...,-0.051850,0.786006,0.669407,0.988720,0.164188,-0.042237,0.478838,0.284957,0.014918,0.596166
4,2020-01-11,4,1.688767,1.123299,2.411143,-1.518019,-2.608174,-0.092445,0.172981,-0.700243,...,-0.053965,0.943710,0.779539,1.154469,0.169794,-0.014473,0.457357,-0.019206,-0.254867,0.203410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037467,2020-11-26,3228,2.863880,2.354470,3.442855,-0.367008,-1.288185,1.237951,-1.062319,-1.936876,...,-0.077145,-0.218845,-0.677157,0.241384,0.066385,-0.106235,0.233775,-0.739699,-0.981184,-0.474419
1037468,2020-11-26,3229,2.193660,1.366086,3.337594,-1.029890,-1.669004,-0.050964,-0.403719,-1.967603,...,-0.655699,0.127605,-0.468057,0.828150,-0.740185,-0.993194,-0.458630,0.147552,-0.153186,0.917622
1037469,2020-11-26,3230,2.997420,1.879952,4.827222,0.063381,-1.229616,1.308801,0.369541,-0.660267,...,-0.563107,-0.703884,-1.031925,-0.244547,0.364359,0.033683,0.590167,-0.308732,-0.452348,-0.056429
1037470,2020-11-26,3231,2.783941,1.074804,4.555070,-1.077723,-1.666085,-0.397469,-0.420119,-1.787267,...,0.145051,0.240206,-0.567698,1.125124,-0.909044,-1.071697,-0.420652,-0.160216,-0.552683,0.257796


In [6]:
df_cams['time'] = pd.to_datetime(df_cams['time'], infer_datetime_format=True)

In [7]:
props = pd.read_csv("../../data/cams/shape_properties.csv")
props

,shapeID,AFFGEOID,ALAND,AWATER,COUNTYFP,COUNTYNS,GEOID,LSAD,NAME,STATEFP
0,0,0500000US21007,639387454,69473325,7,516850,21007,6,Ballard,21
1,1,0500000US21017,750439351,4829777,17,516855,21017,6,Bourbon,21
2,2,0500000US21031,1103571974,13943044,31,516862,21031,6,Butler,21
3,3,0500000US21065,655509930,6516335,65,516879,21065,6,Estill,21
4,4,0500000US21069,902727151,7182793,69,516881,21069,6,Fleming,21
...,...,...,...,...,...,...,...,...,...,...
3227,3228,0500000US31073,1186616237,11831826,73,835858,31073,6,Gosper,31
3228,3229,0500000US39075,1094405866,3695230,75,1074050,39075,6,Holmes,39
3229,3230,0500000US48171,2740719114,9012764,171,1383871,48171,6,Gillespie,48
3230,3231,0500000US55079,625440563,2455383635,79,1581100,55079,6,Milwaukee,55


In [10]:
df_total = pd.merge(props, df_covid, how="inner", left_on=['GEOID'], right_on=['county_id'])
df_total = pd.merge(df_total,df_cams,how="inner", left_on=["date",'shapeID'], right_on=["time",'shapeID'])
df_total = df_total[df_total.columns.drop(["AFFGEOID","ALAND","shapeID","AWATER","COUNTYFP","COUNTYNS","LSAD","NAME","STATEFP","time","county_id"])]
df_total = df_total.rename({"GEOID":"shapeID"},axis=1)
df_total = df_total.sort_values(by=["shapeID","date"])
df_total

,shapeID,date,mobility_transit_stations,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_residential,mobility_workplaces,smoothed_cumul,smoothed_d1,...,x7_amax,x8_mean,x8_amin,x8_amax,x9_mean,x9_amin,x9_amax,x10_mean,x10_amin,x10_amax
12582,10003,2020-03-29,-42.0,-28.0,-6.0,2.0,17.0,-39.0,141.809524,21.892857,...,0.762621,-0.290963,-0.987737,0.032615,-0.376480,-0.833365,0.210322,0.481506,-0.215669,1.328427
12583,10003,2020-03-30,-39.0,-26.0,0.0,49.0,17.0,-40.0,166.000000,23.750000,...,0.860248,0.550598,-0.238915,1.072211,-0.201743,-0.840541,0.407437,0.070182,-0.885137,0.932346
12584,10003,2020-03-31,-40.0,-35.0,-9.0,78.0,11.0,-25.0,191.000000,25.892857,...,1.432990,-0.418361,-1.100802,0.766658,0.034947,-0.495268,0.470400,-0.747065,-1.050835,-0.306901
12585,10003,2020-04-01,-56.0,-38.0,-19.0,17.0,10.0,-30.0,212.238095,38.250000,...,0.858769,-1.154608,-1.715199,-0.941215,0.829711,0.320043,1.218661,-0.731472,-1.216402,0.030832
12586,10003,2020-04-02,-52.0,-31.0,-12.0,-49.0,19.0,-45.0,254.857143,46.392857,...,1.974859,-0.561181,-1.704962,0.777859,0.254482,-0.507295,0.935299,-0.212581,-1.210952,0.418377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29557,55133,2020-11-16,-31.0,-23.0,-9.0,98.0,14.0,-35.0,20912.666667,777.357143,...,0.245586,0.285412,-0.105664,0.863202,-0.217611,-0.801242,0.307856,0.241920,-0.126822,0.736519
29558,55133,2020-11-17,-34.5,-16.0,-1.0,144.0,4.0,-6.0,21829.952381,893.464286,...,-0.812017,0.697624,0.464090,0.950370,0.101097,-0.132406,0.507043,0.568914,0.204780,0.828934
29559,55133,2020-11-18,-38.0,-17.0,-9.0,114.0,3.0,-10.0,23075.476190,906.571429,...,-0.587159,0.753704,0.416389,1.219656,0.332824,-0.363286,0.880683,0.157192,-0.286517,0.703741
29560,55133,2020-11-19,-26.0,-17.0,-3.0,74.0,12.0,-32.0,24195.761905,770.071429,...,-0.314402,0.924094,0.587659,1.672445,0.053305,-0.199086,0.431580,-0.046998,-0.446054,0.140200


In [9]:
df_total.to_csv("../../data_processed/usa_urban_final.csv")